In [1]:
"""
This script applies the fixed-timestep master recession curve watertable fluctuation method (e.g., Heppner and Nimmo 2005) 
to estimate recharge from synthetic hydrographs. The method is implemented on a series of generated 
recharge events, and recharge estimates are compared to known values. The output is the relative error (%) of the 
recharge estimate, calculated as described in Becke et al. (2025).
"""
import os
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from pathlib import Path

# --- Configuration ---
FOLDER_NAME = "FT_MRC"
BASE_DIR = Path.home() / "workspace" / "WTF" / "OutputFiles" 
INPUT_DIR = BASE_DIR / "data" 
OUTPUT_DIR = BASE_DIR / FOLDER_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

sci = [0.1, 0.25, 0.5, 0.75, 0.9]
time = np.linspace(0, 400., 401)

# Event time windows
event_configs = {
    1: {"t1": 30, "t2": 90, "t3": 120},
    2: {"t1": 120, "t2": 210, "t3": 240},
    3: {"t1": 240, "t2": 360, "t3": 390},
}

def average(dataset, interval=1):
    return [(dataset[i] + dataset[i - interval]) / 2 for i in range(interval, len(dataset))]

def fluctuation_rate(dataset, interval=1):
    return [dataset[i - interval] - dataset[i] for i in range(interval, len(dataset))]

def extract_WL(WT_fluc_data, average_EL_data):
    return [b for a, b in zip(WT_fluc_data, average_EL_data) if a > 0]

def extract_recession_rate(WT_fluc_data, average_EL_data):
    return [a for a, b in zip(WT_fluc_data, average_EL_data) if a > 0]

def daily_extend(Z_WT, A, B):
    return [i - (A * i + B) for i in Z_WT]

def Errors_MRC(event, itr, time, t1, t2, t3):
    recharge_empirical_error = []

    time_to_fit = time[t1:t2 + 1]
    time_to_extend = time[t2:t3 + 1]

    for j in range(1, itr + 1):
        try:
            wl_file = INPUT_DIR / f"hydro_{j:04d}.csv"
            var_file = INPUT_DIR / f"var_{j:04d}.csv"

            df_hydrograph = pd.read_csv(wl_file, header=None).to_numpy().flatten()
            df_var = pd.read_csv(var_file, header=None).to_numpy()

            wl = df_hydrograph
            av_WL = average(wl)
            WT_fluc_rate = fluctuation_rate(wl)

            av_WL_ex = np.array(extract_WL(WT_fluc_rate, av_WL)).flatten()
            RR_ex = np.array(extract_recession_rate(WT_fluc_rate, av_WL))

            # Fit linear model
            def linear_function(x, K1, K2):
                return K1 * x + K2

            parameters, _ = curve_fit(linear_function, av_WL_ex, RR_ex, absolute_sigma=True, method='trf')
            K1, K2 = parameters

            # Extend hydrograph
            Z_WT = df_hydrograph[t2 - 1:t3 + 1]
            projected = daily_extend(Z_WT, K1, K2)

            n = min(len(Z_WT), len(projected))
            delta_h_emp = np.sum(Z_WT[1:n] - projected[:n - 1])
            delta_t = time_to_extend[-1] - time_to_fit[-1]

            rech_empirical_est = (delta_h_emp / delta_t) * df_var[3][0]
            true_recharge = df_var[0][0]
            rech_error = ((rech_empirical_est - true_recharge) / true_recharge) * 100

            recharge_empirical_error.append(rech_error)

        except Exception as e:
            print(f"[Warning] Skipping hydrograph {j:04d}: {e}")
            continue

    return recharge_empirical_error

# Run for all 3 events
for event_id, config in event_configs.items():
    errors = Errors_MRC(
        event=event_id,
        itr=1000,  # Change if you have more files
        time=time,
        t1=config["t1"],
        t2=config["t2"],
        t3=config["t3"]
    )

    output_file = OUTPUT_DIR / f"RE{event_id}_MRC_Rerror.csv"
    np.savetxt(output_file, errors, delimiter=',', header=f"MRCR_Rerror_Event{event_id}", comments='')
    print(f"Exported errors for event {event_id}: {output_file}")


Exported errors for event 1: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\FT_MRC\RE1_MRC_Rerror.csv
Exported errors for event 2: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\FT_MRC\RE2_MRC_Rerror.csv
Exported errors for event 3: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\FT_MRC\RE3_MRC_Rerror.csv
